## Создание нового бакета

In [1]:
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 8.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 9.1 MB/s eta 0:00:00


In [2]:
import boto3

s3 = boto3.client(
    's3',
    endpoint_url='http://minio:9000',
    aws_access_key_id='minio',
    aws_secret_access_key='minio-password',
)

bucket_name = "batch-bucket"

existing = [b['Name'] for b in s3.list_buckets()['Buckets']]
if bucket_name not in existing:
    s3.create_bucket(Bucket=bucket_name)
else:
    print(f"{bucket_name} is existing")

## Запись в minio

In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("to_s3")
    .master("spark://spark-master:7077")
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.jars.packages",
            "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262")
    .config("spark.hadoop.fs.s3a.access.key", "minio")
    .config("spark.hadoop.fs.s3a.secret.key", "minio-password")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .getOrCreate()
)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-cb89a882-32ac-4b8e-af9c-da8c85d6fc9e;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
downloading https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.3.4/hadoop-aws-3.3.4.jar ...
	[SUCCESSFUL ] org.apache.hadoop#hadoop-aws;3.3.4!hadoop-aws.jar (353ms)
downloading https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-bundle/1.12.262/aws-java-sdk-bundle-1.12.262.jar ...
	[SUCCESSFUL ] com.amazonaws#aws-java-sdk-bundle;1.12.262!aws-java-sdk-bundle.jar (24964ms)
downloading https://repo1.maven.org/maven2/org/wildfly/openssl/wildfly-openssl/1.0.7.

In [2]:
from pyspark.sql import Row
import random

count = 1000
data = [
    Row(id=i, name=f"test_record_{i}", value=round(random.uniform(0, 1000), 2))
    for i in range(1, count + 1)
]

df = spark.createDataFrame(data)
df.show()

+---+--------------+------+
| id|          name| value|
+---+--------------+------+
|  1| test_record_1|787.86|
|  2| test_record_2|462.91|
|  3| test_record_3|534.92|
|  4| test_record_4|932.76|
|  5| test_record_5|282.25|
|  6| test_record_6| 11.39|
|  7| test_record_7|696.54|
|  8| test_record_8|826.02|
|  9| test_record_9|556.78|
| 10|test_record_10| 22.09|
| 11|test_record_11|868.09|
| 12|test_record_12|564.66|
| 13|test_record_13|511.07|
| 14|test_record_14|581.58|
| 15|test_record_15|681.07|
| 16|test_record_16|934.35|
| 17|test_record_17| 99.32|
| 18|test_record_18|230.19|
| 19|test_record_19|535.66|
| 20|test_record_20|363.93|
+---+--------------+------+
only showing top 20 rows



In [3]:
df.repartition(10).write.mode("overwrite").parquet(f"s3a://batch-bucket/parquet_table/")

25/10/16 15:50:27 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [5]:
df_check = spark.read.parquet(f"s3a://batch-bucket/parquet_table/")
df_check.show(truncate=False)
df_check.count()

+---+---------------+------+
|id |name           |value |
+---+---------------+------+
|371|test_record_371|478.34|
|445|test_record_445|68.44 |
|211|test_record_211|823.56|
|149|test_record_149|312.37|
|86 |test_record_86 |232.7 |
|491|test_record_491|397.39|
|266|test_record_266|790.16|
|3  |test_record_3  |534.92|
|257|test_record_257|860.8 |
|103|test_record_103|684.82|
|110|test_record_110|847.1 |
|68 |test_record_68 |811.67|
|421|test_record_421|671.11|
|364|test_record_364|144.37|
|190|test_record_190|984.4 |
|97 |test_record_97 |120.8 |
|381|test_record_381|58.58 |
|393|test_record_393|221.61|
|438|test_record_438|737.89|
|214|test_record_214|778.84|
+---+---------------+------+
only showing top 20 rows



1000

In [ ]:
spark.stop()